In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
movies_df = pd.read_csv('ml-1m/movies.dat', sep="::", header=None)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
movies_df.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
ratings_df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
movies_df.columns = ["Movie_id", "Movie_title", "Genres"]
ratings_df.columns = ["User_id", "Movie_id","Rating", "Timestamp"]

In [7]:
movies_df.head()

,Movie_id,Movie_title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings_df.head()

,User_id,Movie_id,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
movies_df['List_Index'] = movies_df.index

In [10]:
movies_df.head()

,Movie_id,Movie_title,Genres,List_Index
0,1,Toy Story (1995),Animation|Children's|Comedy,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama,3
4,5,Father of the Bride Part II (1995),Comedy,4


In [11]:
merged_df = movies_df.merge(ratings_df, on="Movie_id")

In [12]:
merged_df = merged_df.drop('Timestamp', axis=1).drop('Genres', axis=1).drop('Movie_title',axis=1)

In [13]:
merged_df.head()

,Movie_id,List_Index,User_id,Rating
0,1,0,1,5
1,1,0,6,4
2,1,0,8,4
3,1,0,9,5
4,1,0,10,5


In [20]:
userGroup = merged_df.groupby("User_id")
userGroup.first().head()

,Movie_id,List_Index,Rating
User_id,,,
1,1,0,5
2,21,20,1
3,104,102,4
4,260,257,5
5,6,5,2


In [35]:
userGroup.groups

{1: Int64Index([     0,  22893,  41541,  67447, 141660, 144754, 158459, 167921,
             168684, 172011, 179963, 185725, 190432, 196749, 222307, 223999,
             233289, 246891, 248460, 249471, 251662, 273611, 294283, 299210,
             310576, 341051, 359307, 370615, 437274, 439731, 482132, 500398,
             508478, 527560, 528890, 550463, 554929, 619091, 625325, 632053,
             635661, 652378, 719607, 721191, 745569, 752653, 756381, 758896,
             783830, 833231, 836694, 853800, 893329],
            dtype='int64'),
 2: Int64Index([  9992,  31493,  34062,  46891,  47810,  62125,  70928,  75785,
              84501,  93971,
             ...
             905131, 907418, 908228, 933864, 947031, 950523, 955640, 965318,
             981016, 991649],
            dtype='int64', length=129),
 3: Int64Index([ 32718,  67448, 129015, 152763, 162460, 165344, 177640, 179281,
             186980, 256513, 262356, 284174, 296221, 299211, 301529, 312854,
             349985, 35

In [37]:
num_Users_training = 2000
train_X = [] 
i = 1
for user_id, user_info in userGroup:
    temp = [0]*len(movies_df)
    for num, movie in user_info.iterrows():
        temp[movie['List_Index']] = movie['Rating']/5.0
    train_X.append(temp)
    i+=1
    if i >= 2000:
        break

In [40]:
len(train_X[0])

3883

In [41]:
hidden_units = 20
visible_units = len(movies_df)
vb = tf.placeholder("float64", [visible_units])
hb = tf.placeholder("float64", [hidden_units])
W = tf.placeholder("float64", [visible_units,hidden_units])

In [44]:
#Input Phase
v0 = tf.placeholder("float64", [None, visible_units])
_h0 = tf.nn.sigmoid(tf.matmul(v0,W)+hb)
h0 = tf.nn.relu(tf.sign(_h0 - tf.random_uniform(tf.shape(_h0),dtype="float64")))

#Reconstruction Phase
_v1 = tf.nn.sigmoid(tf.matmul(h0,tf.transpose(W))+vb)
v1 = tf.nn.relu(tf.sign(_v1- tf.random_uniform(tf.shape(_v1),dtype="float64")))
h1 = tf.nn.sigmoid(tf.matmul(v1,W)+hb)